# 章節 13：模擬

## 13.7 序列關聯性（Serial Correlation）

目前爲止的結果是基於一連串模型的選擇；我們審視這些選擇看哪些是錯誤的來源：

- 轉換線性長度變成體積，我們假設腫瘤大約是球體。這個假設可能對於小的腫瘤還可以，但不適合太大的腫瘤。
- 腫瘤生長率的分佈是用連續的模型去擬合 Zhang 等人提供的 53 為病人的數據。擬合只是近似的，更大量的樣本可能導致不同的分佈。
- 生長的模型沒有考慮腫瘤的亞型跟等級；這假設是跟 Zhang 等人的結論一致：「不同大小、亞型和等級的腎腫瘤的生長速率彼此範圍寬廣並且顯著重疊。」但如果是大樣本，差異可能變明顯。
- 生長率的分佈跟腫瘤的大小無關。這假設可能不真實，非常小跟非常大的腫瘤，成長的限制在於血液的供應。但是 Zhang 等人觀察的腫瘤大小在 1 到 12 公分，而且他們發現大小跟成長率沒有統計顯著關係。所以如果有關係，也可能是很微弱的關係，至少在這個大小區間。
- 在模擬時，成長率在每個區間跟之前的成長率是獨立的。事實上，腫瘤可能會因為過去生長很快，而現在也生長很快。也就是說，生長率是有序列關聯性。

上述，第一跟最後一個是最有問題的假設。這邊會先處理生長率的序列關聯性，再來看球體問題。

為了模擬生長，給定一個Cdf，這邊寫了一個迭代器回傳一個相關聯的序列。下方是演算法的流程：

1. 從高斯分佈產生相關的數值。我們很容易基於前一個數值來計算下一個數值的分佈。
1. 將每個數值轉換成其對應的高斯 CDF。
1. 用高斯 CDF 來取樣

3. Transform each cumulative probability to the corresponding value using the given Cdf.

以下是程式碼的實作：

<pre>
def CorrelatedGenerator(cdf, rho):
    # cdf 是期望的 cdf
    # rho 是期望的相關性

    # 第一個 x 是從一個均值為 0 標準差為 1 的高斯取樣
    x = random.gauss(0, 1)
    
    # Transform 方法轉換成對應的分佈
    # Transform converts them to the desired distribution.
    yield Transform(x)
    
    # 後續的 x 是根據之前的 x ，在均值為 x*rho ，變異數為 1 - rho**2
    sigma = math.sqrt(1 - rho**2);
    while True:
        x = random.gauss(x * rho, sigma)
        yield Transform(x) 
        
        
    # Transform maps from each Gaussian value, x, to a value from the given Cdf, y.        
def Transform(x):

    # GaussianCdf computes the CDF of the standard Gaussian distribution at x, returning a cumulative probability.
    p = thinkbayes.GaussianCdf(x)
    
    #  Cdf.Value maps from a cumulative probability to the corresponding value in cdf.
    y = cdf.Value(p)
    return y

</pre>

根據成長率的 CDF，資訊在轉換的時候可能會流失，所以真實的關聯可能會比 rho 低。例如作者從 rho=0.4 的分佈中產生 100000 數值，實際的關聯為 0.37。但是因為我們正在猜測正確的關聯性，兩個已經夠接近了。

還記得 MakeSequence 用迭代器當作參數。這個介面讓我們用不同的迭代器：

<pre>
iterator = UncorrelatedGenerator(cdf)
# seq1 是沒關聯的序列
seq1 = MakeSequence(iterator)

iterator = CorrelatedGenerator(cdf, rho)
# seq2 是有關聯的序列帶有近似的 rho 參數
seq2 = MakeSequence(iterator)
</pre>

現在我們來看序列關聯的結果；下表是當腫瘤大小為 6 公分時其對應的年齡百分位，分別用沒關聯的迭代器跟 rho=0.4 有關聯的迭代器。

有關聯的迭代器讓生長快的腫瘤更快，慢的生長更慢，所以年齡的涵蓋範圍較大。第 5 百分位差別不到，但是第 95 百分位就差到 6 歲多。

為了更精確的及算百分位，我們需要更好的評估真實的序列關聯性。但是這個模型已經足夠回答一開始的問題：一個 15.5 公分的腫瘤 ，他已經形成超過 8 年的機率是多少？

這邊是程式碼：

<pre>
# class Cache

# 計算某一大小為 cm 的腫瘤超過年齡 age 的機率
def ProbOlder(self, cm, age):
    # cm 是腫瘤的大小
    # age 是腫瘤年齡的門檻（超過多少年齡）
    bucket = CmToBucket(cm)
    cdf = self.ConditionalCdf(bucket)
    p = cdf.Prob(age)
    return 1-p
</pre>

在沒有關聯的情況下，15.5 公分的腫瘤超過 8 年的機率是 0.999，或幾乎是確定的。若是關聯性為 0.4 的情況下，腫瘤生長速度更快的可能性更大，但機率是 0.995。即使關聯性為 0.8，機率是 0.978。

另一個錯誤的來源是假設腫瘤是球體。對一個腫瘤在線性觀察到的是 15.5 x 15 公分的情況下，這假設可能不是有效的。如果假設腫瘤是比較平坦的，這可能與 6 公分的球體的體積差不多。

用較小的體積和 0.8 的關聯性，形成超過 8 年的機率仍然有 95%。所以即使考慮模型錯誤，這麼大的腫瘤幾乎也不可能在診斷之前不到 8 年就已形成。